In [0]:
import ast
import datetime
import queue
import random
from collections import defaultdict
import numpy as np

In [2]:
#For Google Colaboratory
from google.colab import files
uploaded = files.upload()

Saving in_degree_file.csv to in_degree_file.csv
Saving out_degree_file.csv to out_degree_file.csv


In [0]:
def Brandes_Betweenness_Function(G,st,betw_cent):
    Qu = queue.Queue()  # queue --> FIFO
    Stck = []  # stack --> LIFO
    Pred = defaultdict(lambda: [])
    dist = defaultdict(lambda:-1)
    dist[st] = 0
    sigma = defaultdict(lambda: 0)
    sigma[st] = 1
    Qu.put(st)

    while not Qu.empty():
        v = Qu.get()
        if v in Stck:  # prevent a loop back to the visited node
            continue
        Stck.append(v)
        for w in G.get(v, []):
            if dist[w] < 0:
                Qu.put(w)
                dist[w] = dist[v] + 1
            if dist[w] == dist[v] + 1:
                sigma[w] += sigma[v]
                Pred[w].append(v)

    delta = defaultdict(lambda: 0)
    
    while Stck:
        w = Stck.pop()
        for v in Pred[w]:
            delta[v] += sigma[v] / sigma[w] * (1 + delta[w])
        if w != start:
            betw_cent[w] += delta[w]
    return betw_cent


In [4]:
if __name__ == "__main__":

    dfile = "out_degree_file.csv"
    deg_dict = {}
    with open(dfile, 'r')as file:
        for line in file:
            if len(line) > 1:
              deg_dict[int(line.split(":")[0])] = ast.literal_eval(line.split(":")[1])
    source_list = list(deg_dict.keys())
    
    #randomly sample some start nodes
    source_list_r = np.random.choice(source_list, 500)
    print (len(source_list))
    print (len(source_list_r))

    betw_cent = defaultdict(lambda: 0)
    for start in source_list_r:
        begin = datetime.datetime.now()
        betw_cent = Brandes_Betweenness_Function(deg_dict, start,betw_cent)
        #print("start from node :", start,"start:", begin, "end:", datetime.datetime.now())
    
    top_k = 10
    print("the top", top_k, "nodes are:", sorted(betw_cent.items(), key=lambda x: -x[1])[:top_k])


81399
500
the top 10 nodes are: [(0, 31871565.28075648), (1, 3164500.7797233174), (813286, 1833523.3771573403), (5442012, 721607.4836404215), (15527013, 558905.7190127556), (17093617, 508136.18120094354), (79797834, 496158.4495956605), (3829151, 369930.14428156696), (2, 367077.54366154195), (59804598, 365939.6394210596)]
